In [2]:
# Import necessary libraries
import numpy as np
from scipy.stats import norm
import datetime
import yfinance as yf
from math import *


In [3]:
# Cumulative standard normal distribution
def cdf(x):
    return (1.0 + erf(x / sqrt(2.0))) / 2.0

In [4]:

# Intermediate calculation used by both the Bjerksund-Stensland approximations
def phi(s, t, gamma, h, i, rd, b, v):
    lambda1 = (-rd + gamma * b + 0.5 * gamma * (gamma - 1) * v**2) * t
    dd = -(log(s / h) + (b + (gamma - 0.5) * v**2) * t) / (v * sqrt(t))
    k = 2 * b / (v**2) + (2 * gamma - 1)
    try:
        return exp(lambda1) * s**gamma * (cdf(dd) - (i / s)**k * cdf(dd - 2 * log(i / s) / (v * sqrt(t))))
    except OverflowError as err:
        return exp(lambda1) * s**gamma * cdf(dd)


In [5]:
# Call Price based on Bjerksund/Stensland Model
# Parameters
#   underlying_price: Price of underlying asset
#   exercise_price: Exercise price of the option
#   time_in_years: Time to expiration in years
#   rf: Risk-free rate for the foreign currency
#   rd: Risk-free rate for the domestic currency
#   volatility: Volatility percentage

def bjerksund_stensland_call(underlying_price, exercise_price, time_in_years, rf, rd, volatility):
    z = 1
    b = rd - rf
    
    dt = volatility * sqrt(time_in_years)
    drift = b
    v2 = volatility**2
    
    b1 = sqrt((z * drift / v2 - 0.5)**2 + 2 * rd / v2)
    beta = (1 / 2 - z * drift / v2) + b1
    binfinity = beta / (beta - 1) * exercise_price
    bb = max(exercise_price, rd / drift * exercise_price)
    ht = -(z * drift * time_in_years + 2 * dt) * bb / (binfinity - bb)
    i = bb + (binfinity - bb) * (1 - exp(ht))

    if underlying_price < i and beta < 100:
        alpha = (i - exercise_price) * i**(-beta)
        return (alpha * underlying_price**beta 
                - alpha * phi(underlying_price, time_in_years, beta, i, i, rd, drift, volatility) 
                + phi(underlying_price, time_in_years, 1, i, i, rd, drift, volatility) 
                - phi(underlying_price, time_in_years, 1, exercise_price, i, rd, drift, volatility) 
                - exercise_price * phi(underlying_price, time_in_years, 0, i, i, rd, drift, volatility) 
                + exercise_price * phi(underlying_price, time_in_years, 0, exercise_price, i, rd, drift, volatility))
    
    return underlying_price - exercise_price


In [6]:

# Put Price based on Bjerksund/Stensland Model
# Parameters
#   underlying_price: Price of underlying asset
#   exercise_price: Exercise price of the option
#   time_in_years: Time to expiration in years
#   rf: Risk-free rate for the foreign currency
#   rd: Risk-free rate for the domestic currency
#   volatility: Volatility percentage

def bjerksund_stensland_put(underlying_price, exercise_price, time_in_years, rf, rd, volatility):
    z = -1
    b = rd - rf
    
    asset_new = underlying_price
    underlying_price = exercise_price
    exercise_price = asset_new

    dt = volatility * sqrt(time_in_years)
    drift = b
    v2 = volatility**2

    b1 = sqrt((z * drift / v2 - 0.5)**2 + 2 * rd / v2)
    beta = (1 / 2 - z * drift / v2) + b1
    binfinity = beta / (beta - 1) * exercise_price
    bb = max(exercise_price, rd / drift * exercise_price)
    ht = -(z * drift * time_in_years + 2 * dt) * bb / (binfinity - bb)
    i = bb + (binfinity - bb) * (1 - exp(ht))
        
    if underlying_price < i and beta < 100: # To avoid overflow
        alpha = (i - exercise_price) * i**(-beta)
        return (alpha * underlying_price**beta 
                - alpha * phi(underlying_price, time_in_years, beta, i, i, rd, drift, volatility) 
                + phi(underlying_price, time_in_years, 1, i, i, rd, drift, volatility) 
                - phi(underlying_price, time_in_years, 1, exercise_price, i, rd, drift, volatility) 
                - exercise_price * phi(underlying_price, time_in_years, 0, i, i, rd, drift, volatility) 
                + exercise_price * phi(underlying_price, time_in_years, 0, exercise_price, i, rd, drift, volatility))
    
    return underlying_price - exercise_price    

In [17]:

# Black-Scholes price and Greeks for European Call Option
def black_scholes_call(S, K, T, rf, rd, sigma):
    # Calculate d1 and d2
    d1 = (log(S / K) + (rd - rf + 0.5 * sigma ** 2) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    
    # Call option price
    call_price = S * exp(-rf * T) * norm.cdf(d1) - K * exp(-rd * T) * norm.cdf(d2)
    
    # Greeks
    delta = exp(-rf * T) * norm.cdf(d1)
    gamma = exp(-rf * T) * norm.pdf(d1) / (S * sigma * sqrt(T))
    vega = S * exp(-rf * T) * sqrt(T) * norm.pdf(d1)
    rho = K * T * exp(-rd * T) * norm.cdf(d2)
    theta = -S * exp(-rf * T) * norm.pdf(d1) * sigma / (2 * sqrt(T)) - rf * S * exp(-rf * T) * norm.cdf(d1) + rd * K * exp(-rd * T) * norm.cdf(d2)
    
    return call_price, delta, gamma, vega, rho, theta

# Black-Scholes price and Greeks for European Put Option
def black_scholes_put(S, K, T, rf, rd, sigma):
    # Calculate d1 and d2
    d1 = (log(S / K) + (rd - rf + 0.5 * sigma ** 2) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    
    # Put option price
    put_price = K * exp(-rd * T) * norm.cdf(-d2) - S * exp(-rf * T) * norm.cdf(-d1)
    
    # Greeks
    delta = -exp(-rf * T) * norm.cdf(-d1)
    gamma = exp(-rf * T) * norm.pdf(d1) / (S * sigma * sqrt(T))
    vega = S * exp(-rf * T) * sqrt(T) * norm.pdf(d1)
    rho = -K * T * exp(-rd * T) * norm.cdf(-d2)
    theta = -S * exp(-rf * T) * norm.pdf(d1) * sigma / (2 * sqrt(T)) + rf * S * exp(-rf * T) * norm.cdf(-d1) - rd * K * exp(-rd * T) * norm.cdf(-d2)
    
    return put_price, delta, gamma, vega, rho, theta


bs_call_price, call_delta, call_gamma, call_vega, call_rho, call_theta = black_scholes_call(S, K, T, rf, rd, sigma)
    
    # Calculate Black-Scholes price and Greeks for put
bs_put_price, put_delta, put_gamma, put_vega, put_rho, put_theta = black_scholes_put(S, K, T, rf, rd, sigma)
    
    # Print results
print(f"Black-Scholes Call Price: {bs_call_price:.2f}")
print(f"Call Delta: {call_delta:.2f}, Gamma: {call_gamma:.2f}, Vega: {call_vega:.2f}, Rho: {call_rho:.2f}, Theta: {call_theta:.2f}")
    
print(f"Black-Scholes Put Price: {bs_put_price:.2f}")
print(f"Put Delta: {put_delta:.2f}, Gamma: {put_gamma:.2f}, Vega: {put_vega:.2f}, Rho: {put_rho:.2f}, Theta: {put_theta:.2f}")
    

Black-Scholes Call Price: 0.33
Call Delta: 0.29, Gamma: 0.19, Vega: 20.09, Rho: 12.14, Theta: -0.32
Black-Scholes Put Price: 1.25
Put Delta: -0.68, Gamma: 0.19, Vega: 20.09, Rho: -29.06, Theta: -1.37


In [15]:
# Example usage
if __name__ == "__main__":
    # Assume the inputs for the example
    S = 83.54  # Current USD/INR exchange rate
    K = 85 # Strike price
    T = 0.5  # Time to maturity in years (6 months)
    rf = 0.05  # Risk-free interest rate for the foreign currency (2%)
    rd = 0.0622  # Risk-free interest rate for the domestic currency (5%)
    sigma = 0.03  # Volatility (10%)
    
    # Calculate option price
    call_price = bjerksund_stensland_call(S, K, T, rf, rd, sigma)
    put_price = bjerksund_stensland_put(S, K, T, rf, rd, sigma)
    
    # Print results
    print(f"American Call Option Price (USD/INR): {call_price:.2f}")
    print(f"American Put Option Price (USD/INR): {put_price:.2f}")


American Call Option Price (USD/INR): 0.33
American Put Option Price (USD/INR): 2.04
